In [2]:
import pandas as pd
import re
import json
import argparse
#from hanspell import spell_checker
from tqdm import tqdm
import html
import time

In [3]:
def remove_by_account(file_path, account_id):
    with open(file_path, 'r', encoding='UTF8') as f:
        accounts = f.read()
    accounts = [i for i in accounts.split('\n') if i]
    accounts = list(map(lambda x: x.split(' '), accounts))
    ids, name = zip(*accounts)
    
    if account_id in ids:
        return True
    return False

In [4]:
def is_keyword_in(file_path, text):
    with open(file_path, 'r', encoding='UTF8') as f:
        keyword = f.read()
    keyword = keyword.split('\n')
    for k in keyword:
        if k in text:
            return True
    return False

In [5]:
def replace_text(file_path, text):
    with open(file_path, 'r', encoding='UTF8') as f:
        replace = f.read()
    replace = [i for i in replace.split('\n') if i]
    replace = list(map(lambda x: x.split(','), replace))

    refined = html.unescape(text)
    refined = refined.replace('xa0', ' ')
    # refined = refined.replace('u200b', ' ')
    refined = refined.replace('&39;', "'")
    refined = re.sub(r'([U|u]0001f[a-zA-Z0-9]{3})|([U|u]200d)', '', refined)
    refined = re.sub(r'@[a-zA-z0-9_]+', '', refined)
    refined = re.sub(r'\[[^)]*?\]|\<[^)]*?\>|\([^)]*?\)', '', refined)
    refined = re.sub(r'\A(RT :) *', '', refined)
    refined = re.sub(r'http[^ ]*', '', refined)

    refined = refined.lower()
    for old, new in replace:
        refined = refined.replace(old.strip(), new.strip())

    return ' '.join(refined.split())


In [6]:
path = './'
news_keyword_path = path + 'txt/' + 'news_keyword.txt'
ads_keyword_path = path + 'txt/' + 'ads_keyword.txt'
unrelated_keyword_path = path + 'txt/' + 'unrelated_keyword.txt'
account_path = path + 'txt/' + 'account.txt'
replace_path = path + 'txt/' + 'replace.txt'

In [7]:
with open('./all.json', 'r', encoding='UTF8') as f:
        data = json.load(f)

In [8]:
ret = 0
act = 0
news = 0
ads = 0
unr = 0

df = pd.DataFrame(columns=['date', 'time', 'twit', 'refined_twit', 'twit_id'])
    # err = 0; normal = 0
for i in tqdm(range(len(data))):
    dummy = data[i]
    if dummy['tag'] == 1: #리트윗 데이터이면 삭제
        ret +=1
        pass
    elif remove_by_account(account_path, dummy['author_id']):
        act +=1
        pass
    elif is_keyword_in(news_keyword_path, dummy['twit']):
        news +=1
        pass
    elif is_keyword_in(ads_keyword_path, dummy['twit']):
        ads+=1
        pass
    elif is_keyword_in(unrelated_keyword_path, dummy['twit']):
        unr+=1
        pass
    else:
        refined = dummy['twit']
        refined = replace_text(replace_path, refined)
        refined = re.sub('[-=+,#/\:@*\"※ㆍ』‘|\(\)\[\]`\'…》\”\“\’·$&]', '', refined)
        #refined = spelled_text(refined)
        #time.sleep(1)
        refined = ' '.join(refined.split())
        df = df.append({'date': dummy['date'], 'time':dummy['time'].split('.')[0], 'twit' : dummy['twit'], 'refined_twit' : refined ,'twit_id' : dummy['twit_id'],}, ignore_index=True)
        #print(df)
        #break

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1820235/1820235 [1:38:27<00:00, 308.12it/s]


In [12]:
#print("num twits before remove duplicates ",len(df))
df = df.drop_duplicates(['twit_id'], keep='first').reset_index(drop=True)
df.to_feather('../all.ftr')
print('Pre-processing completed')

Pre-processing completed


In [9]:
print("retwit ", ret)
print("account ", act)
print("news ", news)
print("advertise ",ads)
print("unrelated ",unr)

retwit  1389673
account  12846
news  141815
advertise  596
unrelated  28338


In [13]:
print((ret+act+news+ads+unr))

1573268


In [20]:
print((len(data)-(ret+act+news+ads+unr)) -len(df))

49452


In [14]:
df = df.drop_duplicates(['twit_id'], keep='first').reset_index(drop=True)

In [18]:
df = df.drop_duplicates(subset=['twit'])

In [10]:
print(len(data)-(ret+act+news+ads+unr))

246967


In [19]:
print(len(df))

197515


In [12]:
df

,date,time,twit,refined_twit,twit_id
0,2022-07-30,22:40:01,백신 4차 노바백스로 맞음맞으면서 부작용 사례 물어보니아직 접수 된 건이 하나도 없...,백신 4차 노바백스로 맞음맞으면서 부작용 사례 물어보니아직 접수 된 건이 하나도 없...,1553510754581487616
1,2022-07-30,08:16:57,노바백스 후기화이자 보다 안 아픔 근데 팔 아파서 잘 못 움직임 근데 이것도 케바케...,노바백스 후기화이자 보다 안 아픔 근데 팔 아파서 잘 못 움직임 근데 이것도 케바케...,1553293555854061571
2,2022-07-29,04:17:49,@haruobear 오오 저 이거 너무 궁굼했어요!!!!!!!흑 4차는 저도 노바백...,오오 저 이거 너무 궁굼했어요!!!!!!!흑 4차는 저도 노바백스 맞으려고 했는데 ...,1552870990924812289
3,2022-07-29,03:27:33,@haruobear 노바백스 맞았어여??? 헉헉 괜찮으신가여?,노바백스 맞았어요??? 헉헉 괜찮으신가여?,1552858339389779968
4,2022-07-28,06:22:50,"1,2차 화이자3차 노바백스생명의 위협을 느껴서 이제서야 3차 맞음근데 15분 있다...",12차 화이자3차 노바백스생명의 위협을 느껴서 이제서야 3차 맞음근데 15분 있다가...,1552540061635293184
...,...,...,...,...,...
246962,2022-08-02,23:53:42,노바백스 있으려나?,노바백스 있으려나?,1554616461640867841
246963,2022-08-02,07:14:52,4차 접종 했다. 그동안 화이자가 너무 힘들었다 했더니 바꿔보재서 노바백스를 선택했...,4차 접종 했다. 그동안 화이자가 너무 힘들었다 했더니 바꿔보재서 노바백스를 선택했...,1554365096557551616
246964,2022-08-02,03:05:14,출발 일주일 전에 4차 노바백스 백신을 맞았던 것이 신의 한수였다. 이 어린양을 바...,출발 일주일 전에 4차 노바백스을 맞았던 것이 신의 한수였다. 이 어린양을 바른 길...,1554302273509023750
246965,2022-08-01,17:03:45,@lsy_piece @youmoo00 노바백스가 부작용이 젤 적다는데 3가지 맞아본...,노바백스가 부작용이 젤 적다는데 3가지 맞아본 중 제일 아무지도 않네요미세한 미열과...,1554150908258586627
